Modifying Question Paper

In [ ]:
pip install opencv-python


In [4651]:
pip install xlwt

Note: you may need to restart the kernel to use updated packages.


In [225]:
import xlwt
import os
from commonfunctions import *
import numpy as np
import pandas as pd
import cv2
import skimage.io as io
import matplotlib.pyplot as plt
from skimage.color import rgb2gray
from skimage.morphology import binary_erosion, binary_dilation, binary_closing,skeletonize, thin
from skimage.measure import find_contours
from skimage.draw import rectangle
from skimage.transform import resize


# Show the figures / plots inside the notebook
%matplotlib inline
%load_ext autoreload
%autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


get BinaryImg:

input: 

grey img

threshold

output:

binary image

In [226]:
def getBinaryImg(img, thresh):

    binary_img = img.copy()
    binary_img[binary_img > thresh] = 1
    binary_img[binary_img <= thresh] = 0
    return binary_img

getClosedImg:

input:

binary img - window size

output:

closed image

In [227]:
def getClosedImg(img, win_size):
    closed_img = binary_dilation(binary_erosion(img, footprint=win_size), footprint=win_size)
    return closed_img


getAspectRatioContour

gets image contour depending on aspect ratio 

In [228]:
def getAspectRatioContour(img, thresh = 0.7, win_size =  np.ones((2, 2), dtype=int), lower_aspect = 6, upper_aspect = 10):
    binary_img = getBinaryImg(img, thresh)
    
    closed_img = getClosedImg(binary_img, win_size)
    
    #show_images([closed_img])
    
    contour = find_contours(closed_img, 0.8)
    #print(contour)
    bounding_rects = []
    
    for rect in contour:
        x_min = rect[:, 1].min()
        x_max = rect[:, 1].max()
        y_min = rect[:, 0].min()
        y_max = rect[:, 0].max()
        
        aspect_ratio = (x_max - x_min) / (y_max - y_min) 
        area = (x_max - x_min) * (y_max - y_min) 
        if (lower_aspect < aspect_ratio < upper_aspect) and (300 < area < 9000):
            bounding_rects.append([int(x_min), int(x_max), int(y_min), int(y_max)])
    bounding_rects = sorted(bounding_rects, key = lambda item: (item[2] + 2*item[0]))
    return bounding_rects


getRectContour:

input: grey img, threshold, window size

output: array of contours with area as the first element and sorted according to area

In [229]:
def getRectContour(img, thresh = 0.7, win_size =  np.ones((2, 2), dtype=int), inverse = 0):
    binary_img = getBinaryImg(img, thresh)

    closed_img = getClosedImg(binary_img, win_size)
    #show_images([closed_img])
    contour = find_contours(closed_img, 0.8)

    bounding_rects = []
    area = []
    
    for rect in contour:
        x_min = rect[:, 1].min()
        x_max = rect[:, 1].max()
        y_min = rect[:, 0].min()
        y_max = rect[:, 0].max()
        
        area = (x_max - x_min)*(y_max - y_min)
        
        aspect_ratio = (x_max - x_min) / (y_max - y_min) 
        
        if aspect_ratio > 1:
            bounding_rects.append([int(area),int(x_min), int(x_max), int(y_min), int(y_max)])
    
    bounding_rects = sorted(bounding_rects, reverse = True)


    return bounding_rects


getCVRectContour

input: grey img, upper thresh, lower thresh, sigma, windowsize

output: array of contours sorted according to area

In [230]:
def getCVRectContour(img, ut = 150,lt = 100,kernel = 3, win_size =  np.ones((3, 3), dtype=int)):

    img_blur = cv2.GaussianBlur(img, (kernel, kernel), 7)

    img_canny = cv2.Canny(img_blur,lt,ut) 

    img_dial = cv2.dilate(img_canny, win_size, iterations=2)
    img_closed = cv2.erode(img_dial, win_size, iterations=1)
    
    #show_images([img_blur, img_closed])
    contours, heirarchy = cv2.findContours(img_closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    bounding_rects = []
    
    for rect in contours:
        area = cv2.contourArea(rect)
        #print(area)
        perimeter = cv2.arcLength(rect, True)
        points = cv2.approxPolyDP(rect, 0.02*perimeter, True)
        #print(points)
        
        if (len(points) == 4) and (area > 50):
            bounding_rects.append(rect)
            
    bounding_rects = sorted(bounding_rects, key = cv2.contourArea, reverse = True)    
    
    return bounding_rects
    
#image = cv2.cvtColor(io.imread('1/c.jpg'), cv2.COLOR_BGR2GRAY)

#c = getCVRectContour(image)



In [231]:
def getAspectCornerPoints(contour, i,img):
    c = contour[i]

    points = np.array([[c[0], c[2]],
                      [c[1], c[2]],
                      [c[1], c[3]],
                      [c[0], c[3]]])
    
    #grey_img = img.copy()
    #plt.plot([points[0][0], points[1][0]], [points[0][1], points[1][1]], color = "black", linewidth = 1)
    #plt.plot([points[1][0], points[2][0]], [points[1][1], points[2][1]], color = "black", linewidth = 1)
    #plt.plot([points[2][0], points[3][0]], [points[2][1], points[3][1]], color = "black", linewidth = 1)
    #plt.plot([points[3][0], points[0][0]], [points[3][1], points[0][1]], color = "black", linewidth = 1)
    #plt.imshow(grey_img)
    #plt.show()
    
    
    return points

getCVCornerPoints

input: CVcontour

output: cornerpoints

In [232]:
def getCVCornerPoints(contour):
    perimeter = cv2.arcLength(contour, True)
    points = cv2.approxPolyDP(contour, 0.02*perimeter, True)
    return points
#p = getCVCornerPoints(c[0])


getCornerPoints

input: contour, index of the largest contour area, image

output: corner points

In [233]:
def getCornerPoints(contour, i,img):
    c = contour[i][1:5]
    #print(c)
    points = np.array([[c[0], c[2]],
                      [c[1], c[2]],
                      [c[1], c[3]],
                      [c[0], c[3]]])
    #print(img)
    grey_img = img.copy()
    #plt.plot([points[0][0], points[1][0]], [points[0][1], points[1][1]], color = "black", linewidth = 1)
    #plt.plot([points[1][0], points[2][0]], [points[1][1], points[2][1]], color = "black", linewidth = 1)
    #plt.plot([points[2][0], points[3][0]], [points[2][1], points[3][1]], color = "black", linewidth = 1)
    #plt.plot([points[3][0], points[0][0]], [points[3][1], points[0][1]], color = "black", linewidth = 1)
    #plt.imshow(grey_img)
    #plt.show()
    

    
    #show_images([grey_img])
    #print(x1)
    return points



   

getOrderedCornerPoints

input: points

output: ordered points

In [234]:
def getOrderedCornerPoints(points):
    points = points.reshape((4,2))
    points_ordered = np.zeros((4,1,2), np.int32)
    s = points.sum(1)
    diff = np.diff(points, axis = 1)
    #print(points)
    points_ordered[0] = points[np.argmin(s)]
    points_ordered[1] = points[np.argmin(diff)]
    points_ordered[2] = points[np.argmax(diff)]
    points_ordered[3] = points[np.argmax(s)]
    #print(points_ordered)
    return points_ordered

#widthImg = image.shape[0]
#heightImg = image.shape[1]
#
#points = getOrderedCornerPoints(p)
#
#p2 = np.float32([[0,0], [widthImg,0], [0,heightImg], [widthImg, heightImg]])
#p1 = np.float32(points)
#
#img = cv2.getPerspectiveTransform(p1, p2)
#img_warp = cv2.warpPerspective(image,img, (widthImg, heightImg))
#img_warp = resize(img_warp, (800, 1000))
#show_images([img_warp])
#print(img_warp)

getIDRegion:

input: warped image - index depending on size of ID

output: ID image

In [235]:
def getIDRegion(img, index = 1):

    i = img.copy()
    i[i > 0.6] = 1
    i[i <= 0.6] = 0
    i = i[20:250, 80:560]

    win_size = np.ones((18, 42), dtype=int)
    
    c = getRectContour(i, thresh = 0.7, win_size =  win_size)
    p = getCornerPoints(c, index,i)
    p_ordered = getOrderedCornerPoints(p)
    p2 = np.float32([[0,0], [300,0], [0,100], [300, 100]])
    p1 = np.float32(p_ordered)
    
    ID_img = cv2.getPerspectiveTransform(p1, p2)
    ID_warp = cv2.warpPerspective(i,ID_img, (300, 100))
    
    return ID_warp
        
#ID = getIDRegion(img_warp, index = 2)
#show_images([ID])

getQuestionRegion

In [236]:
def getQuestionRegion(img, index = 1, mcq = 4):

    I = img.copy()
    I[I > 0.55] = 1
    I[I <= 0.55] = 0

    if mcq == 2:
        win_size = np.ones((3, 15), dtype=int)
        c = getAspectRatioContour(I, thresh = 0.5, win_size =  win_size, lower_aspect = 3.5, upper_aspect = 4.5)
    elif mcq == 3:
        win_size = np.ones((4, 20), dtype=int)
        c = getAspectRatioContour(I, thresh = 0.5, win_size =  win_size, lower_aspect = 5.5, upper_aspect = 7)
    elif mcq == 4:
        win_size = np.ones((2, 20), dtype=int)
        c = getAspectRatioContour(I, thresh = 0.5, win_size =  win_size, lower_aspect = 7, upper_aspect = 10)
    elif mcq == 5:
        win_size = np.ones((4, 20), dtype=int)
        c = getAspectRatioContour(I, thresh = 0.5, win_size =  win_size, lower_aspect = 10, upper_aspect = 20)
    #print(c)
    p = getAspectCornerPoints(c, index, I)
    
    p_ordered = getOrderedCornerPoints(p)
    
    p2 = np.float32([[0,0], [300,0], [0,100], [300, 100]])
    p1 = np.float32(p_ordered)
    
    Q_img = cv2.getPerspectiveTransform(p1, p2)
    Q_warp = cv2.warpPerspective(I,Q_img, (300, 100))
  
    #show_images([Q_warp])
    return Q_warp

#Q = getQuestionRegion(img_warp, index = 12, mcq = 3)

In [237]:
def splitrows(img, rowsnum):
    rows=[]
    Rsections=(100/(rowsnum+1))
    extra=Rsections/3
    extra=int(extra)
    Rsections=int(Rsections)
    Rbegin=int(Rsections+extra)
    Rend=int(Rbegin+Rsections)
    
    for i in range (rowsnum):
        rows.insert(i,img[Rbegin:Rend][:])
        Rbegin=Rend
        Rend=Rbegin+Rsections
    return rows

def splitcolumns(img):
    columns=[]
    sections=30
    begin=0
    end=begin+sections
    
    for i in range (10):
        columns.insert(i,img[:,begin:end])
        begin=end
        end=begin+sections
    return columns


def getbubble(img,rowsnum):
    bubbles=[]
    j=0
    k=0
    for i in range (rowsnum*10):
        IDrows = splitrows(img,rowsnum)
        IDrow = IDrows[j]
        IDcolumns = splitcolumns(IDrow)
        IDcolumn = IDcolumns[k]
        bubbles.insert(i,IDcolumn)
        k=k+1
        if (k>9):
            j=j+1
            k=0
    return bubbles


#rowsnum =2
#IDbubbles = getbubble(ID,rowsnum)
#bubblenum = IDbubbles[0]
#show_images([bubblenum])

In [285]:
def getID(rowsnum, IDbubbles):
    countC=0
    countR=0
    errorID=0
    sumarray=[]
    avgid=[]
    errorarr=[]
    sumID=0
    i=0
    pixelval=np.zeros((rowsnum,10))
    for images in IDbubbles:
        totalpixels=cv2.countNonZero(images)
        pixelval[countR][countC]=totalpixels
        sumID=sumID+totalpixels
        countC+=1
        if(countC == 10):
            sumarray.insert(i,sumID)
            sumID=0
            countR+=1
            countC=0
            i+=1
    
    for i in range (rowsnum):
        avgid.insert(i,int(sumarray[i]/10))
        errorarr.insert(i,int(sumarray[i]/120))
    
    #print(pixelval)
    #print(avgid)
    
    for j in range (rowsnum):
        for i in range (10):
            if(pixelval[j][i]<(avgid[j]-errorarr[j])):
                errorID=errorID+1
                

    
    arrindex=[]
    index=[]
    
    for x in range (0,rowsnum):
        arr=pixelval[x]
        index=np.where(arr==np.amin(arr))
        arrindex.append(index[0][0])
        
    if errorID>(rowsnum):
        arrindex = "wrong id"

    return arrindex

In [295]:
def convert(list):
    if list != "wrong id":
        s = [str(i) for i in list]
        res = int("".join(s))
    else:
        res = "wrong id"
    return(res)

#IDfinal=[]
#IDfinal.insert(0,int(convert(ID)))


In [287]:
def splitQcolumns(img, colsnum):
    cols=[]
    Csections=(300/(colsnum))
    Csections=int(Csections)
    Cbegin=0
    Cend=Cbegin+Csections
    
    for i in range (colsnum):
        cols.insert(i,img[:,Cbegin:Cend])
        Cbegin=Cend
        Cend=Cbegin+Csections
    return cols

#colsnum=3
#columns=splitQcolumns(Q,colsnum)
#column=columns[0]
#show_images([column])

In [288]:
my_data = np.loadtxt('1.txt', delimiter=",", unpack=True)
list2= my_data[:]
#print(list2)

In [289]:
def getQanswer(colsnum,columns):   
    countC=0
    countR=0
    error=0
    summ=0
    errorarr = [50]
    pixelval=np.zeros((1,colsnum))
    for images in columns:
        totalpixels=cv2.countNonZero(images)
        summ=summ+totalpixels
        pixelval[countR][countC]=totalpixels
        countC+=1
        if(countC == 10):countR+=1;countC=0
    
    avg=summ/colsnum
    
    for i in range (colsnum):
        if(pixelval[0][i]<(avg-200)):
            error=error+1
    
    arrindex=[]
    index=[]
    for x in range (0,1):
        arr=pixelval[x]
        index=np.where(arr==np.amin(arr))
        arrindex.append(index[0][0])
    
    if error>1:
        return(errorarr)
    else:
        return(arrindex)
    
#answer=getQanswer(colsnum,columns)
#print(answer)

In [290]:

quesnum=13

def getScore(mcq,quesnum,img, list2):
    score=0
    quesarr=[]
    for i in range (quesnum): 
        ques=getQuestionRegion(img, index = i,mcq = mcq)
        columns=splitQcolumns(ques,mcq)
        answer1=getQanswer(mcq,columns)
        answer=answer1[0]+1
        if(answer==list2[i]):
            score=score+1
            quesarr.insert(i,1)
        else:
            quesarr.insert(i,0)
    return quesarr, score

#quesarr, score = getScore(quesnum)

#print(score)
#print(quesarr)

In [291]:
#cell_list=quesarr
#arr = np.array(cell_list)
#dataframe = pd.DataFrame(arr)
#print(dataframe)
#data = dataframe.style.set_properties(align="left")
## Converting it in a excel-file
#data.to_excel(r"book1.xlsx",index=None,header="code")    

In [293]:
folder = '2'
files = os.listdir(folder)
paths = folder + '/' + files[0]
images = []
for file in range(len(files)):
    path = folder + '/' + files[file]
    images.append(cv2.cvtColor(io.imread(path), cv2.COLOR_BGR2GRAY))  
    
    
ID_1 = [2,1,2,2,1,1,2,1,1,1,2,2,1,1,2]
ID_2 = [1,1,1,1,1,1,0,1]
ID_3 = [1,1]
ID_4 = [1,1,1,1,1,1,1,1,1,2,2,1,1]
mcq = 2
my_data = np.loadtxt('2.txt', delimiter=",", unpack=True)
list2= my_data[:]


imgs_warp = []
for image in range(len(images)):
    c = getCVRectContour(images[image])
    p = getCVCornerPoints(c[0])
    
    widthImg = images[image].shape[0]
    heightImg = images[image].shape[1]
    
    points = getOrderedCornerPoints(p)
    p2 = np.float32([[0,0], [widthImg,0], [0,heightImg], [widthImg, heightImg]])
    p1 = np.float32(points)
    
    img = cv2.getPerspectiveTransform(p1, p2)
    img_warp1 = cv2.warpPerspective(images[image],img, (widthImg, heightImg))
    imgs_warp.append(resize(img_warp1, (800, 1000)))

    
ID_regs = []
for img_warp in range(len(imgs_warp)):
    if folder == '1':
        ID_regs.append(getIDRegion(imgs_warp[img_warp], index = ID_1[img_warp]))
        rowsnum = 2
    elif folder == '2':
        ID_regs.append(getIDRegion(imgs_warp[img_warp], index = ID_2[img_warp]))
        rowsnum = 4
    elif folder == '3':
        ID_regs.append(getIDRegion(imgs_warp[img_warp], index = ID_3[img_warp]))
        rowsnum = 4
    elif folder == '4':
        rowsnum = 7
        ID_regs.append(getIDRegion(imgs_warp[img_warp], index = ID_4[img_warp]))

#show_images(ID_regs)

IDbubbles = []
for ID_reg in range(len(ID_regs)):
    IDbubbles.append(getbubble(ID_regs[ID_reg], rowsnum))

IDfinal=[]
for IDbubble in range(len(IDbubbles)):
    ID = getID(rowsnum, IDbubbles[IDbubble])
    IDfinal1=[]
    IDfinal1.insert(0,(convert(ID)))
    IDfinal.append(IDfinal1)
    
#print(IDfinal)
cell_list=IDfinal
arr = np.array(cell_list)
dataframe = pd.DataFrame(arr)
data = dataframe.style.set_properties(align="left")
data.to_excel(r"book1.xlsx",index=None,header="code")

s = []
q = []
for img_warp in range(len(imgs_warp)):
    score=0
    quesarr=[]
    for i in range (quesnum): 
        Q=getQuestionRegion(imgs_warp[img_warp], index = i,mcq = mcq)
        #show_images([ques])
        Qbubbles = splitQcolumns(Q, mcq)
        #show_images(Qbubbles)
        answer1=getQanswer(mcq,Qbubbles)
        answer=answer1[0]+1
        if(answer==list2[i]):
            score=score+1
            quesarr.insert(i,1)
        else:
            quesarr.insert(i,0)
    #print(quesarr)
    s.insert(img_warp,score)
    q.insert(img_warp,quesarr)



data1=IDfinal
data2=q
data3=s
df = pd.DataFrame({'ID': data1,
                   'Questions': data2,
                  'Score' : data3})
df.to_excel(r"book1.xlsx",startrow=1,index=False,header=False)



C:\Users\larar\AppData\Local\Temp\ipykernel_25452\120716004.py:19: RuntimeWarning: divide by zero encountered in double_scalars
  aspect_ratio = (x_max - x_min) / (y_max - y_min)
